In [1]:
import sys, os
sys.path.append('/work')
from train.utils.create_model import create_unet
import yaml
import torchinfo

In [2]:
with open('/work/train/segmentation/train_config/segmentation_culane.yaml', 'r') as f:
    train_config = yaml.load(f, Loader=yaml.Loader)
train_config

{'train': {'loss': {'loss_type': 'lane_dice',
   'lane_dice': {'num_cls': 5},
   'cross_entropy': {'weights': 'None',
    'ignore_index': -100,
    'reduction': 'mean',
    'label_smoothing': 0.0},
   'dice': {'weights': 'None', 'num_cls': 5}},
  'optimizer': {'optimizer_type': 'adamw',
   'adamw': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': '1e-8',
    'weight_decay': 0.01,
    'amsgrad': False,
    'maximize': False,
    'foreach': 'None',
    'capturable': False,
    'differentialable': False,
    'fused': 'None'},
   'adam': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': '1e-8',
    'weight_decay': 0.01,
    'amsgrad': False,
    'foreach': 'None',
    'maximize': False,
    'capturaable': False,
    'differentialable': False,
    'fused': 'None'},
   'rmsprop': {'lr': 0.0006,
    'alpha': 0.99,
    'eps': '1e-8',
    'weight_decay': 0,
    'momentum': 0,
    'centered': False,
    'capturable': False,
    'foreach': 'None',
    'maximize': False,
    'differentialab

In [34]:
import regex
def load_yaml(path):
    # traverse nested dictionary with the path to the value (hierarchical key set)
    def traverse_leaves_with_keys(d, parent_key=()):
        for key, value in d.items():
            current_path = parent_key + (key, )
            if isinstance(value, dict):
                yield from traverse_leaves_with_keys(value, current_path)
            else:
                yield current_path, value
    
    def get_list_depth(lst):
        if not isinstance(lst, list):
            return 0
        if not lst:
            return 1
        return 1 + max(get_list_depth(item) for item in lst)
    
    def convert_innermost_lists_to_tuples(obj):
        if isinstance(obj, list):
            if not any(isinstance(item, list) for item in obj):
                return tuple(obj)
            
            return [convert_innermost_lists_to_tuples(item) for item in obj]
        return obj
        
    with open(path, 'r') as fr:
        configs = yaml.load(fr, Loader=yaml.Loader)
    
    for path, leaf in traverse_leaves_with_keys(configs):
        root = configs
        # we need to access to the node that precedes the terminal node
        for i in range(len(path)-1):
            root = root[path[i]]
            
        parent_key = path[-1]
        value = root[parent_key]
        
        p = r'\d{0,1}e[+,-]\d{0,5}'
        p = regex.compile(p)
        # define the rules to filter the yaml value
        if value == 'None': # convert string 'None' to python None
            root[parent_key] = None
        elif type(value)==str and len(p.findall(value)) != 0: # convert string scientific notation to float value
            root[parent_key] = float(value)
        elif isinstance(value, list): # convert innermost lists to tuples
            depth = get_list_depth(value)
            if depth >= 2: # if depth of the list is bigger than 2
                root[parent_key] = convert_innermost_lists_to_tuples(value)
        
    return configs

load_yaml('/work/train/segmentation/train_config/segmentation_culane.yaml')


{'train': {'loss': {'loss_type': 'lane_dice',
   'lane_dice': {'num_cls': 5},
   'cross_entropy': {'weights': None,
    'ignore_index': -100,
    'reduction': 'mean',
    'label_smoothing': 0.0},
   'dice': {'weights': None, 'num_cls': 5}},
  'optimizer': {'optimizer_type': 'adamw',
   'adamw': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': 1e-08,
    'weight_decay': 0.01,
    'amsgrad': False,
    'maximize': False,
    'foreach': None,
    'capturable': False,
    'differentialable': False,
    'fused': None},
   'adam': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': 1e-08,
    'weight_decay': 0.01,
    'amsgrad': False,
    'foreach': None,
    'maximize': False,
    'capturaable': False,
    'differentialable': False,
    'fused': None},
   'rmsprop': {'lr': 0.0006,
    'alpha': 0.99,
    'eps': 1e-08,
    'weight_decay': 0,
    'momentum': 0,
    'centered': False,
    'capturable': False,
    'foreach': None,
    'maximize': False,
    'differentialable': False},
   '

In [ ]:
with open('/work/train/segmentation/model_config/unet.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
unet = create_unet('/work/train/segmentation/model_config/unet.yaml')
torchinfo.summary(unet, (1, 3, 224, 672)) # torchsummary takes some gpu memory (if only loaded with cpu device, no gpu memory is used)

Layer (type:depth-idx)                                  Output Shape              Param #
UNet                                                    [1, 5, 224, 672]          --
├─Encoder: 1-1                                          [1, 384, 7, 21]           --
│    └─Stem: 2-1                                        [1, 48, 56, 168]          --
│    │    └─Conv2d: 3-1                                 [1, 48, 56, 168]          2,352
│    └─ModuleList: 2-2                                  --                        --
│    │    └─Stage: 3-2                                  [1, 48, 56, 168]          2,780,793
│    │    └─Stage: 3-3                                  [1, 96, 28, 84]           1,634,994
│    │    └─Stage: 3-4                                  [1, 192, 14, 42]          3,489,480
│    │    └─Stage: 3-5                                  [1, 384, 7, 21]           4,653,000
├─Decoder: 1-2                                          [1, 48, 224, 672]         --
│    └─ModuleList: 2-3       